In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import csv
data={}
test_data = {}
for i in range(1,101):
    with open('simulationData\sim_'+str(i)+'.csv', 'r' ) as theFile:
        reader = csv.reader(theFile)
        headers = next(reader, None)
        actions = []
        states = []
        rewards = []
        for line in reader:
            actions.append(int(line[3]))
            newState = np.zeros(13)
            newState[int(line[8])-1] = 1
            newState[int(line[9])-1] = 1
            states.append(newState)
            rewards.append(int(line[5]))
        block = [
                {
                    'action': np.array([actions])-1,
                    'state': np.array([states]),
                    'reward': np.array([rewards]),
                    'id': 'S'+str(i),

                    'block': 0
                }

            ]
        if i<91:
            data['S'+str(i)] = block
        else:
            test_data['S'+str(i)] = block

In [3]:
data

{'S1': [{'action': array([[0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
           0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1,
           0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
           0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
           0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
           0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
           1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
           0, 0, 0, 0, 0, 1]]),
   'state': array([[[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           ...,
           [0., 1., 0., ..., 1., 0., 0.],
           [0., 1., 0., ..., 0., 0., 0.],
           [0., 1., 0., ..., 0., 0., 0.]]]),
   'reward': array([[0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
           1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=13, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/exploreExploit_stateVector/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-07-12 15:35:59,205 - DeepRL - DEBUG - version control: (None, None)
2021-07-12 15:35:59,207 - DeepRL - DEBUG - learning rate: 0.01
2021-07-12 15:35:59,209 - DeepRL - DEBUG - global iters: 50
2021-07-12 15:35:59,210 - DeepRL - DEBUG - training data-points: 90
2021-07-12 15:35:59,212 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-07-12 15:36:00,165 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/exploreExploit_stateVector/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-12 15:36:09,155 - DeepRL - DEBUG - global iter =    0 total obj: 9985.8359
2021-07-12 15:36:16,826 - DeepRL - DEBUG - global iter =    1 total obj: 9969.7951
2021-07-12 15:36:25,412 - DeepRL - DEBUG - global iter =    2 total obj: 9949.5382
2021-07-12 15:36:35,591 - DeepRL - DEBUG - global iter =    3 total obj: 9917.0888
2021-07-12 15:36:46,120 - DeepRL - DEBUG - global iter =    4 total obj: 9884.5121
2021-07-12 15:36:56,451 - DeepRL - DEBUG - global iter =    5 total obj: 9852.7591
2021-07-12 15:37:07,740 - DeepRL - DEBUG - global iter =    6 total obj: 9822.0315
2021-07-12 15:37:16,704 - DeepRL - DEBUG - global iter =    7 total obj: 9789.4002
2021-07-12 15:37:25,113 - DeepRL - DEBUG - global iter =    8 total obj: 9742.4743
2021-07-12 15:37:33,494 - DeepRL - DEBUG - global iter =    9 total obj: 9693.8086
2021-07-12 15:37:42,522 - DeepRL - DEBUG - global iter =   10 total obj: 9645.8788
2021-07-12 15:37:53,264 - DeepRL - DEBUG - global iter =   11 total obj: 9594.6661
2021

INFO:tensorflow:../results/exploreExploit_stateVector/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

{'merged': [{'reward': array([[0, 1, 1, ..., 1, 1, 0],
          [0, 0, 1, ..., 0, 0, 1],
          [0, 1, 0, ..., 0, 1, 1],
          ...,
          [0, 1, 1, ..., 0, 0, 1],
          [1, 0, 0, ..., 0, 1, 0],
          [0, 0, 0, ..., 0, 1, 1]]),
   'action': array([[1, 0, 0, ..., 1, 1, 0],
          [0, 1, 1, ..., 0, 1, 1],
          [0, 0, 0, ..., 1, 0, 1],
          ...,
          [1, 0, 0, ..., 0, 1, 0],
          [0, 0, 1, ..., 0, 0, 0],
          [0, 1, 0, ..., 0, 0, 1]]),
   'state': array([[[0., 1., 0., ..., 0., 0., 0.],
           [0., 1., 0., ..., 0., 0., 0.],
           [0., 1., 0., ..., 0., 0., 0.],
           ...,
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
   
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           ...,
           [0., 1., 0., ..., 1., 0., 0.],
           [0., 1., 0., ..., 0., 0., 0.],
        

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-07-12 15:47:28,901 - DeepRL - DEBUG - version control: (None, None)
2021-07-12 15:47:28,903 - DeepRL - DEBUG - learning rate: 0.01
2021-07-12 15:47:28,905 - DeepRL - DEBUG - global iters: 50
2021-07-12 15:47:28,907 - DeepRL - DEBUG - training data-points: 90
2021-07-12 15:47:28,909 - DeepRL - DEBUG - test data-points: 10
2021-07-12 15:47:53,198 - DeepRL - DEBUG - opt started...
2021-07-12 15:47:53,200 - DeepRL - DEBUG - started testing...
2021-07-12 15:48:31,530 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/exploreExploit_stateVector/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-12 15:49:00,943 - DeepRL - DEBUG - global iter =    0 total obj: 9985.0075
2021-07-12 15:49:14,163 - DeepRL - DEBUG - global iter =    1 total obj: 9972.6470
2021-07-12 15:49:27,882 - DeepRL - DEBUG - global iter =    2 total obj: 9956.9417
2021-07-12 15:49:41,418 - DeepRL - DEBUG - global iter =    3 total obj: 9928.3705
2021-07-12 15:49:54,289 - DeepRL - DEBUG - global iter =    4 total obj: 9893.3874
2021-07-12 15:50:07,720 - DeepRL - DEBUG - global iter =    5 total obj: 9849.9797
2021-07-12 15:50:20,933 - DeepRL - DEBUG - global iter =    6 total obj: 9801.3492
2021-07-12 15:50:34,116 - DeepRL - DEBUG - global iter =    7 total obj: 9747.1780
2021-07-12 15:50:47,393 - DeepRL - DEBUG - global iter =    8 total obj: 9701.4318
2021-07-12 15:50:58,147 - DeepRL - DEBUG - global iter =    9 total obj: 9641.9144
2021-07-12 15:50:58,149 - DeepRL - DEBUG - started testing...
2021-07-12 15:51:22,352 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/exploreExploit_stateVector/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-12 15:51:43,079 - DeepRL - DEBUG - global iter =   10 total obj: 9572.3772
2021-07-12 15:51:55,691 - DeepRL - DEBUG - global iter =   11 total obj: 9530.6044
2021-07-12 15:52:08,759 - DeepRL - DEBUG - global iter =   12 total obj: 9482.5973
2021-07-12 15:52:21,907 - DeepRL - DEBUG - global iter =   13 total obj: 9402.8729
2021-07-12 15:52:34,225 - DeepRL - DEBUG - global iter =   14 total obj: 9353.1603
2021-07-12 15:52:44,754 - DeepRL - DEBUG - global iter =   15 total obj: 9254.4111
2021-07-12 15:52:57,693 - DeepRL - DEBUG - global iter =   16 total obj: 9182.4064
2021-07-12 15:53:08,470 - DeepRL - DEBUG - global iter =   17 total obj: 9185.2455
2021-07-12 15:53:18,680 - DeepRL - DEBUG - global iter =   18 total obj: 9204.3739
2021-07-12 15:53:28,728 - DeepRL - DEBUG - global iter =   19 total obj: 9076.6324
2021-07-12 15:53:28,730 - DeepRL - DEBUG - started testing...
2021-07-12 15:53:56,506 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/exploreExploit_stateVector/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-12 15:57:10,903 - DeepRL - DEBUG - global iter =   30 total obj: 8629.1353
2021-07-12 15:57:25,011 - DeepRL - DEBUG - global iter =   31 total obj: 8645.7540
2021-07-12 15:57:41,289 - DeepRL - DEBUG - global iter =   32 total obj: 8615.1291
2021-07-12 15:57:55,031 - DeepRL - DEBUG - global iter =   33 total obj: 8540.9526
2021-07-12 15:58:08,125 - DeepRL - DEBUG - global iter =   34 total obj: 8419.9340
2021-07-12 15:58:19,230 - DeepRL - DEBUG - global iter =   35 total obj: 8380.1871
2021-07-12 15:58:28,240 - DeepRL - DEBUG - global iter =   36 total obj: 8443.4848
2021-07-12 15:58:38,132 - DeepRL - DEBUG - global iter =   37 total obj: 8484.9680
2021-07-12 15:58:49,061 - DeepRL - DEBUG - global iter =   38 total obj: 8344.6791
2021-07-12 15:59:04,603 - DeepRL - DEBUG - global iter =   39 total obj: 8326.4374
2021-07-12 15:59:04,605 - DeepRL - DEBUG - started testing...
2021-07-12 15:59:43,135 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/exploreExploit_stateVector/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-12 16:00:05,029 - DeepRL - DEBUG - global iter =   40 total obj: 8262.6303
2021-07-12 16:00:19,975 - DeepRL - DEBUG - global iter =   41 total obj: 8302.6148
2021-07-12 16:00:34,130 - DeepRL - DEBUG - global iter =   42 total obj: 8330.1299
2021-07-12 16:00:46,824 - DeepRL - DEBUG - global iter =   43 total obj: 8207.9477
2021-07-12 16:00:56,844 - DeepRL - DEBUG - global iter =   44 total obj: 8250.9841
2021-07-12 16:01:05,799 - DeepRL - DEBUG - global iter =   45 total obj: 8287.1384
2021-07-12 16:01:16,042 - DeepRL - DEBUG - global iter =   46 total obj: 8269.0360
2021-07-12 16:01:29,932 - DeepRL - DEBUG - global iter =   47 total obj: 8276.4114
2021-07-12 16:01:45,089 - DeepRL - DEBUG - global iter =   48 total obj: 8243.8648
2021-07-12 16:02:00,369 - DeepRL - DEBUG - global iter =   49 total obj: 8196.0006
2021-07-12 16:02:00,371 - DeepRL - DEBUG - opt finished.
2021-07-12 16:02:00,373 - DeepRL - DEBUG - started testing...
2021-07-12 16:02:39,322 - DeepRL - DEBUG - finished

INFO:tensorflow:../results/exploreExploit_stateVector/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
test_data["S100"]

[{'action': array([[0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
          0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
          0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
          1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
          0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
          1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
          1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
          1, 0, 0, 1, 1, 0]]),
  'state': array([[[1., 1., 0., ..., 0., 0., 0.],
          [1., 1., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          ...,
          [1., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 1.],
          [0., 0., 0., ..., 0., 0., 1.]]]),
  'reward': array([[0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
          0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0